In [0]:
from io import StringIO

In [0]:
business = sc.textFile("/FileStore/tables/business.csv")
review = sc.textFile("/FileStore/tables/review.csv")
user = sc.textFile("/FileStore/tables/user.csv")

## Question 3

In [0]:
b3 = business.map(lambda x: x.split("::"))

In [0]:
stanford = b3.filter(lambda x: 'stanford'.lower() in x[1].lower()).collect()
stanford_business_ids = [b[0] for b in stanford] # get business ids for businesses in stanford

In [0]:
r3 = review.map(lambda x: x.split("::")).filter(lambda x: x[2] in stanford_business_ids) # get reviews with businesses in stanford

In [0]:
r3.take(5)

Out[6]: [['8rd8rQRD3q4Q-Vp83XyQPg',
  'c4mkYDTjJc-_lOM32YaPPQ',
  'pTTUqKB3TxaW43ElWl6aOQ',
  '4.0'],
 ['6g9IS-BRMt6RuUeAFSOwvg',
  'bozvcy7kl2kJ2mBdKVxyUg',
  '1UdClU8UuEvGmDVVCAfmFg',
  '4.0'],
 ['reCJCEvh6D7Zj8R0fifGyA',
  'oCls9DEDz5JXYLOxM-57Dw',
  'rPEBinN7NEn8uosusvBarg',
  '3.0'],
 ['ayDKaRov7rH-S-1ZFeHBkw',
  'w8_PFQ72PYxYc8ZF53aPJA',
  'ce0OlLhdpoxyMQE03Q98lQ',
  '4.0'],
 ['DNSldK4zkLGHv1vLu5r3Fw',
  'C7AngbgK1l4oaK3A6_ylfQ',
  'BXNa3b28Lm8zUXXhMUtSig',
  '3.0']]

In [0]:

user_rating_count = {}
for u, _ in r3.flatMap(lambda x: [(x[1], x[3])]).collect():
    if user not in user_rating_count:
        user_rating_count[u] = 1
    else:
        user_rating_count[u] += 1

In [0]:
user_ratings = r3.flatMap(lambda x: [(x[1], x[3])]).groupByKey().mapValues(lambda x: sum([float(i) for i in x]) / len(x))
user_ratings.take(10)

Out[8]: [('c4mkYDTjJc-_lOM32YaPPQ', 3.411764705882353),
 ('udG2EZW-tXZpNKthkYUE4w', 1.0),
 ('-WfY2TAjaIe0OaCxj0-cFA', 3.5),
 ('JdM7iuxhptOVd5P2J40rZg', 1.0),
 ('WTnwAd9QPcIuOLsp0DpSCw', 5.0),
 ('tK9HNA3tBIbfym7FinuoKw', 1.0),
 ('P5bUL3Engv-2z6kKohB6qQ', 3.625),
 ('Wxd-EKF2Z4RHZUmwQ6Eeew', 3.2777777777777777),
 ('b-RcJ-n6RqLrZfOziYmFOw', 1.0),
 ('H3tPoj8yqmhZfkiol8F1yQ', 3.3333333333333335)]

In [0]:
u3 = user.map(lambda x: x.split("::")).filter(lambda x: x[0] in user_rating_count)

In [0]:
user_names = {}
for user_id, name, _ in u3.collect():
    user_names[user_id] = name

In [0]:
for ur in user_ratings.collect():
    print(ur[0], user_names[ur[0]], ur[1])

c4mkYDTjJc-_lOM32YaPPQ Nicole A. 3.411764705882353
udG2EZW-tXZpNKthkYUE4w Jae M. 1.0
-WfY2TAjaIe0OaCxj0-cFA Heidi M. 3.5
JdM7iuxhptOVd5P2J40rZg A G. 1.0
WTnwAd9QPcIuOLsp0DpSCw mike r. 5.0
tK9HNA3tBIbfym7FinuoKw jane d. 1.0
P5bUL3Engv-2z6kKohB6qQ kim n. 3.625
Wxd-EKF2Z4RHZUmwQ6Eeew Adrienne M. 3.2777777777777777
b-RcJ-n6RqLrZfOziYmFOw April A. 1.0
H3tPoj8yqmhZfkiol8F1yQ Jimmy C. 3.3333333333333335
1k6Yuk4Ww0LiXknsAe5-2g Amberlene W. 4.333333333333333
OdqqWMexTw65RYOWJjCH_Q Giselle G. 3.0
XrVdBKCCQD7FPD9D3QEKtQ Jenny H. 3.5
SEiPIOK0o-kZEpAejAyp_g Monica M. 5.0
fSR_UX9kSeU4oOkrP4zacg Andrew W. 2.5
yVxVGS68r_IDNIFWTK0JTw Grace C. 4.4
pQ254dv_0oyi0VHU1KDkZQ Krista H. 5.0
ANJ-jY5M8k-FKd7kr7x93w Brent A. 3.8
gzpfljP4vcIf3UbLK9hEHg Angelica N. 4.0
f_w0FtaBzHg3BP8WygR4Sg Cindy M. 4.0
Qpmco1Cpe0NLyqhjm_PlzA Christine N. 2.0
xV7P3uCsdLqi3ylzUQegAw Cristine I. 4.0
v0S0EwKSYs3Hdm67C1H3Ng M D. 3.0
dz7zZGkA_T7SJZf9zN2ziA Phuong H. 1.0
5afmpCP3uo2Q02iECWTiNQ Valerie A. 4.0
WS4-QjVF_3QzQ4L-imhdFA Ailee

In [0]:
# outputting to file

with StringIO() as file:
        
    for ur in user_ratings.collect():
        
        name = user_names[ur[0]]
        avg_rating = ur[1]
        
        output = f"{name}\t{avg_rating}\n"
        
        file.write(output)
        
    dbutils.fs.put("/FileStore/tables/Hw2_Q3_output.txt", file.getvalue(), True)

Wrote 72657 bytes.


## Question 4

In [0]:
b4 = business.map(lambda x: x.split("::"))
r4 = review.map(lambda x: x.split("::"))

In [0]:
# bid_categories = b4.map(lambda x: (x[0], x[2][5:-1]))
bid_categories = b4.map(lambda x: (x[0], [s.strip() for s in x[2][5:-1].split(',')]))
bid_categories.take(10)

Out[13]: [('Iu-oeVzv8ZgP18NIB0UMqg', ['Medical Centers', 'Health and Medical']),
 ('ae5tm46-ewAhUJ6HR4fXFA', ['Massage', 'Beauty and Spas']),
 ('lb9qELwduFgBo408x2VKOA', ['Tattoo', 'Beauty and Spas']),
 ('HPWmjuivv3xJ279qSVfNaQ',
  ['Music & DVDs', 'Books', 'Mags', 'Music and Video', 'Shopping']),
 ('DARh6yjrGGVZ7R3Gdh4CMA', ['Food', 'Coffee & Tea']),
 ('ZKcVHVlD-kwmXFlnPzxpMg',
  ['Property Management', 'Home Services', 'Real Estate']),
 ('HIPGr2gSEN4T73tjz47hpw',
  ['Pubs', 'Bars', 'American (Traditional)', 'Nightlife', 'Restaurants']),
 ('A_Fm4v2-gQuGBBI-GBx4Uw', ['Pizza', 'Restaurants']),
 ('-ilTnXu41RrxQITuolQhmQ', ['Food', 'Coffee & Tea']),
 ('L5ABFgTmN-T-uZjuGpoi3Q', ['Food', 'Specialty Food', 'Health Markets'])]

In [0]:
bid_category = bid_categories.flatMap(lambda x: [(s.strip(), x[0]) for s in x[1]])
bid_category.take(100)

Out[14]: [('Medical Centers', 'Iu-oeVzv8ZgP18NIB0UMqg'),
 ('Health and Medical', 'Iu-oeVzv8ZgP18NIB0UMqg'),
 ('Massage', 'ae5tm46-ewAhUJ6HR4fXFA'),
 ('Beauty and Spas', 'ae5tm46-ewAhUJ6HR4fXFA'),
 ('Tattoo', 'lb9qELwduFgBo408x2VKOA'),
 ('Beauty and Spas', 'lb9qELwduFgBo408x2VKOA'),
 ('Music & DVDs', 'HPWmjuivv3xJ279qSVfNaQ'),
 ('Books', 'HPWmjuivv3xJ279qSVfNaQ'),
 ('Mags', 'HPWmjuivv3xJ279qSVfNaQ'),
 ('Music and Video', 'HPWmjuivv3xJ279qSVfNaQ'),
 ('Shopping', 'HPWmjuivv3xJ279qSVfNaQ'),
 ('Food', 'DARh6yjrGGVZ7R3Gdh4CMA'),
 ('Coffee & Tea', 'DARh6yjrGGVZ7R3Gdh4CMA'),
 ('Property Management', 'ZKcVHVlD-kwmXFlnPzxpMg'),
 ('Home Services', 'ZKcVHVlD-kwmXFlnPzxpMg'),
 ('Real Estate', 'ZKcVHVlD-kwmXFlnPzxpMg'),
 ('Pubs', 'HIPGr2gSEN4T73tjz47hpw'),
 ('Bars', 'HIPGr2gSEN4T73tjz47hpw'),
 ('American (Traditional)', 'HIPGr2gSEN4T73tjz47hpw'),
 ('Nightlife', 'HIPGr2gSEN4T73tjz47hpw'),
 ('Restaurants', 'HIPGr2gSEN4T73tjz47hpw'),
 ('Pizza', 'A_Fm4v2-gQuGBBI-GBx4Uw'),
 ('Restaurants', 'A_Fm4v2-gQuGB

In [0]:
bid_category.groupByKey().map(lambda x: (x[0], list(x[1]))).collectAsMap()['']

Out[15]: ['CrjZoiaRdFivE1esbkQ7sw',
 'cvvFEEEkZNyDbNy8UB6esw',
 'RU7bd3h2f6pBlf8BfAyxGQ',
 'wz3GFJrL92IqJWCE9v-BKA',
 'Hy2JzrPyqqHk7ed2g-zWbg',
 'oH0S5GrCTOv_btOQd0UrLA',
 'tnwOUxWjrQeloEsdEzb53Q',
 '15YzmE7eR42WLsKCoWtOTg',
 'Zb22gjD3m_MwFRB6Ay-QOw',
 'IzY2Dgq9VG9L_odPVkzQqg',
 'DGiylpvVZytoSIqSsVH5HQ',
 'I20o1h6gEPTl_dY1dgwxXg',
 'rqn5hFLMJVHaQvfu2Czl5A',
 'H-lXyHlSlaLBOepN_Qvm8w',
 'cZoY24As9-CPqqaj3Nv91g',
 '9GCuUQec-Ugxh_dXSMPbfQ',
 'V0Qqc0uHXvcN3PCxTawGMQ',
 'QtXP6bCfwJEvLoHjJb7mSg',
 '4iTRjN_uAdAb7_YZDVHJdg',
 'WNn8UfSVYrfksEJSjTciZw',
 'CaAFCP3njTjs2AO20p4Ldg',
 'PcngXr5t49QK8Lj1Xi3aIw',
 'AaLBZaMtW9EmhIY3dHRrNA',
 'lBA7uNsdZiw3I3ZJ3bgVMA',
 'DbbdYLsXjr2zSfe98gnPtw',
 'M1-3lbWDPV-crfnkapFMwg',
 'of2INzyV6kuSUWsmK9Vj2w',
 'AcK2QL8tM6y9abwFjqDXWQ',
 'TZqSbTyzsVVcHR9K2fUvcQ',
 'GS3vqosK_1vROEnqzvUrMg',
 'bET_It-VRCKr1Y-q4wgQcg',
 '-4IoLSPumLbEX_2VKbT81g',
 'H4b7mZm90HzNez-ISrtChw',
 'pZXKss_sPkDWJSe6r6kItQ',
 't_vpekiWx01z3biPzdDQUQ',
 'Y1CVNxjODuemVc8bOfGiuw',
 'ZMrku7LqVSeqPLADt

In [0]:
bid_rating = r4.map(lambda x: (x[2], x[3])).groupByKey().map(lambda x: (x[0], [len(list(x[1])), sum(float(i) for i in list(x[1]))])).collectAsMap()
bid_rating

Out[16]: {'qw5gR8vW7mSOK4VROSwdMA': [238, 977.0],
 'Hnz1_h_D1eHSRtQqHSCZkw': [66, 201.0],
 'ZRm8fSEBn8DsSLD4o7T4hQ': [132, 472.0],
 'Q0t7EEaygJ2hb4uFn79owQ': [62, 216.0],
 'fcdjnsgO8Z5LthXUx3y-lA': [103, 342.0],
 'jTPr0dFk2JgGYdDiIGro3Q': [93, 336.0],
 'V9DXseGJNzLeCxUpMZuUnw': [21, 77.0],
 'piXuRfZ81xFGA64WFJrKkQ': [2634, 11539.0],
 'YWSsD10TzzqE1htRG9fQKw': [38, 79.0],
 't6qBeLsoKOUmUwJCTFyjuQ': [73, 279.0],
 'bCXtHSr8a7tnk2DwsodgyQ': [56, 168.0],
 '3DtugUXX4WHOK2T34ZexXg': [2, 2.0],
 '8V1ceChgSzcGUV6jC8CvyQ': [323, 1075.0],
 '5OJ2xEaVY04LCNA6MWg2lg': [28, 82.0],
 '6otX06BtZ5uOvTM7_VjF3A': [44, 174.0],
 '4EljqY21fLOtLaoWCJlsIQ': [22, 79.0],
 '83vp-VLzGCBGyQ3_XOg7Yw': [46, 205.0],
 'HffgoaAdup4MuPjqGZbycg': [31, 107.0],
 'q27xEJjmTVLH4GywA1DmUg': [95, 338.0],
 'hTp23z3Q5PCvuGvKTIQtXw': [218, 709.0],
 'PFppNi9rZWAw5f8e5J41MQ': [463, 1830.0],
 'joP8geCs-JMQr2tmM_f1sw': [81, 314.0],
 'GUoVncYpFzfxt2HJZFBqDw': [28, 92.0],
 'nCWZR9NRc2grAwP1jq33PA': [33, 133.0],
 'j0lgvR3AAG1FL6qj9AmWEg': 

In [0]:
def get_review_count_and_avg(line):
    cat = line[0]
    bids = line[1]
    
    review_sum = 0
    review_count = 0
    
    for bid in bids:
        if bid in bid_rating:
            review_count += bid_rating[bid][0]
            review_sum += bid_rating[bid][1]
        
    review_avg = review_sum / review_count
    
    return (cat, review_count, review_avg)

In [0]:
cat_review_count_avg = bid_category.groupByKey().map(lambda x: (x[0], list(x[1]))).map(get_review_count_and_avg)

In [0]:
for line in cat_review_count_avg.take(5):
        cat = line[0]
        count = line[1]
        rating_avg = line[2]
        
        if cat == '': continue
        
        output = f"{cat}\t{count}\t{rating_avg}\n"
        
        print(output)

Medical Centers	1278	3.4131455399061035

Massage	2732	3.9963396778916547

Beauty and Spas	23404	3.8536147667065457

Books	8848	4.183092224231465

Mags	8848	4.183092224231465



In [0]:
# outputting to file

with StringIO() as file:
        
    for line in cat_review_count_avg.collect():
        cat = line[0]
        count = line[1]
        rating_avg = line[2]
        
        if cat == '': continue
        
        output = f"{cat}\t{count}\t{rating_avg}\n"
        
        file.write(output)
        
    dbutils.fs.put("/FileStore/tables/Hw2_Q4_output.txt", file.getvalue(), True)

Wrote 17711 bytes.


## Question 5

In [0]:
u5 = user.map(lambda x: x.split("::")).map(lambda x: (x[0], x[1])).collectAsMap()
# u5.take(10)
u5

Out[20]: {'JkeCKyEaQlbLd9uZYl4DjA': 'LiiLii C.',
 'cs91PAsv6esdWAaSkzm2lg': 'Jan Ellen T.',
 'cMgGj2FXHEbzdNZdLN_EwA': 'Saki U.',
 'KXJbnHT4PDS1JZNCFKdmMg': 'stephanie h.',
 'Tpmvufw1eea1DrjLAY2jLg': 'Theodore J.',
 'L22W35q3Ci3TytpA2LW34g': 'Doug H.',
 'xiJ6e1qN3Sqv4dS4D8CpNg': 'Amelia M.',
 'pu96s510jutWeFOuofgY2g': 'Ty G.',
 'zcOlcoYhVgEgWxRpTVjUJA': 'Steve K.',
 '_NH7Cpq3qZkByP5xR4gXog': 'Chris M.',
 '9YLxlEqpEjiugcA1NCbQnw': 'Christina W.',
 'PiSHysV8QdhgzU7QIRn1Kg': 'Cheri W.',
 'PB3OGUgRSajFx18EBUwwEQ': 'Lord H.',
 'e9ATa_PlOWiyTS4QhdsJKA': 'Derek F.',
 'urQ0TUF3uhc-oOHzb3F5tQ': 'Yuan D.',
 'DUJLlGEHUerY8feKhjO85A': 'Ryan H.',
 'ti5uWqAxf7pOkKs7csMlDA': 'Tiffany R.',
 'JssCk4of-CQ7j81ZLrZMZg': 'Jennifer C.',
 'dMEMCWkbmi2hzr24_9J-ZA': 'Michael P.',
 'Y_v3O9q_-vSOk25cOEjIkQ': 'Dan S.',
 '7RsdY4_1Bb_bCf5ZbK6tyQ': 'cat u.',
 'oNNAypob-Pj7j7qF68DQ7A': 'Greg A.',
 'SUsFQqcuTl7yYkW_Ic_XLw': 'Ryan S.',
 'MyuQW9aJ1-54tIOQCeUiMw': 'Richard L.',
 'b1OmuM_0FwDbWMs3XWJrAg': 'Square R.',
 'K

In [0]:
r5 = review.map(lambda x: x.split("::")).map(lambda x: (x[1], x[0]))
r5.take(10)

# for r in r5.take(10):
#     print(u5[r[0]], list(r[1]))

Out[21]: [('kT43SxDgMGzbeXpO51f0hQ', '0xuZfa0t4MNWd3eIFF02ug'),
 ('T9hGHsbJW9Hw1cJAlIAWmw', 'bBqVqhOvNgFs8I1Wk68QUQ'),
 ('Z_WAxc4RUpKp3y12BH1bEg', 'fu7TcxnAOdnbdLcyFhMmZg'),
 ('OlMjqqzWZUv2-62CSqKq_A', 'LMy8UOKOeh0b9qrz-s1fQA'),
 ('fs5bpfk-2pvq2v8S1De5pQ', 'JjyRj9EiBXQTFDQAxRtt4g'),
 ('jm0kgXVEdcQ_tQNzsoNXRw', 'Zht8FVW5a9wfU3u2_dmypQ'),
 ('1IzWxAfxuHTnzKOupUOB5Q', '85TbS2RT5f6kqZ5l7_jfRw'),
 ('n_zEixNdHIh9bwEfjXWykg', 'OdShAEdAwR3LiJoyWAfQuQ'),
 ('bkIgi_PJyr6-2doVwu_ojw', 'VSD3I0x_Dz0XC78ifd9A6Q'),
 ('Hjpn4meBXbjJHd8bkG4fSA', 'lyp372NPuGfL6KOsnFGFxw')]

In [0]:
total_reviews = len(r5.collect())

In [0]:
users_review_count = r5.groupByKey().map(lambda r: (u5[r[0]], ((float(len(list(r[1]))) / total_reviews))))
users_review_count.take(10)

Out[23]: [('Caroline A.', 6.05930239251555e-06),
 ('Rhea S.', 3.029651196257775e-06),
 ('William S.', 9.088953588773325e-06),
 ('Jay D.', 2.42372095700622e-05),
 ('Chiquita y.', 1.21186047850311e-05),
 ('Sam C.', 3.029651196257775e-06),
 ('Brian W.', 6.05930239251555e-05),
 ('Yelena Y.', 3.332616315883552e-05),
 ('Nicole A.', 7.877093110270215e-05),
 ('Justin B.', 3.029651196257775e-05)]

In [0]:
top_10 = users_review_count.takeOrdered(10, key=lambda x: -x[1])
for i in top_10:
    print(i[0], "{:4%}".format(i[1]))

A T. 0.078165%
John L. 0.072712%
William J. 0.059987%
David N. 0.049989%
Christina G. 0.049989%
Lisa W. 0.049080%
Jess L. 0.043021%
Melissa M. 0.042112%
Ligaya T. 0.041506%
Mae S. 0.041203%


In [0]:
# outputting to file

with StringIO() as file:
        
    for i in top_10:
        name = i[0]
        percentage = "{:4%}".format(i[1])
        
        output = f"{name}\t{percentage}\n"
        
        file.write(output)
        
    dbutils.fs.put("/FileStore/tables/Hw2_Q5_output.txt", file.getvalue(), True)

Wrote 190 bytes.
